In [0]:
cat_type="Industry"
cat_filter="Clothing/Shoe/Accessory Stores"
interest_date_from, interest_date_to = "2019-1-1", "2019-4-1"
reference_date_from, referece_date_to = "2018-10-1", "2019-1-1"

<br>
## Setup
Includes much of the code from "8. Keywords.ipynb"



In [0]:
from datetime import datetime
interest_date_from, interest_date_to = datetime.strptime(interest_date_from, '%Y-%m-%d'), datetime.strptime(interest_date_to, '%Y-%m-%d')
reference_date_from, referece_date_to = datetime.strptime(reference_date_from, '%Y-%m-%d'), datetime.strptime(referece_date_to, '%Y-%m-%d')

##### Bring in Data

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pprint
import nltk
nltk.download('punkt')

import re
import pandas as pd
import numpy as np
import pickle
import os
from IPython.display import clear_output
clear_output()

In [0]:
data_directory = "/content/gdrive/My Drive/DATS6450 - Labadie - Data/"

from google.colab import drive
drive.mount('/content/gdrive')

if os.path.exists(data_directory+"Pickles/metadata.pkl"):
    pfile = open(data_directory+"Pickles/metadata.pkl", "rb")
    df_metadata = pickle.load(pfile)                 
    pfile.close()

if os.path.exists(data_directory+"Pickles/text_paragraph.pkl"):
    pfile = open(data_directory+"Pickles/text_paragraph.pkl", "rb")
    df_text_prepremarks = pickle.load(pfile)                 
    pfile.close()

clear_output()

In [0]:
! pip install summa
from summa import summarizer
clear_output()

##### Segment Calls by Quarter

In [6]:
from datetime import datetime
from dateutil.parser import parse

now = datetime.now()
today = datetime(now.year, now.month, now.day)
apr_2019 = datetime(2019,4,1)
jan_2019 = datetime(2019,1,1)
oct_2018 = datetime(2018,10,1)
jul_2018 = datetime(2018,7,1)
apr_2018 = datetime(2018,4,1)
jan_2018 = datetime(2018,1,1)
oct_2017 = datetime(2017,10,1)

q4_2017_calls = df_metadata[df_metadata.apply(lambda row: (jan_2018 > row["call date"] >= oct_2017), axis=1)]
q1_2018_calls = df_metadata[df_metadata.apply(lambda row: (apr_2018 > row["call date"] >= jan_2018), axis=1)]
q2_2018_calls = df_metadata[df_metadata.apply(lambda row: (jul_2018 > row["call date"] >= apr_2018), axis=1)]
q3_2018_calls = df_metadata[df_metadata.apply(lambda row: (oct_2018 > row["call date"] >= jul_2018), axis=1)]
q4_2018_calls = df_metadata[df_metadata.apply(lambda row: (jan_2019 > row["call date"] >= oct_2018), axis=1)]
q1_2019_calls = df_metadata[df_metadata.apply(lambda row: (row["call date"] >= jan_2019), axis=1)]

print("Q4 2017 Calls:",q4_2017_calls.shape)
print("Q1 2018 Calls:",q1_2018_calls.shape)
print("Q2 2018 Calls:",q2_2018_calls.shape)
print("Q3 2018 Calls:",q3_2018_calls.shape)
print("Q4 2018 Calls:",q4_2018_calls.shape)
print("Q1 2019 Calls:",q1_2019_calls.shape)

Q4 2017 Calls: (137, 16)
Q1 2018 Calls: (306, 16)
Q2 2018 Calls: (355, 16)
Q3 2018 Calls: (546, 16)
Q4 2018 Calls: (1404, 16)
Q1 2019 Calls: (2328, 16)


In [7]:
docs = df_metadata["Prepared Remarks Text"]
def get_vocab(docs):
    vectorizer = CountVectorizer(binary=True, lowercase=True)
    vectors = vectorizer.fit_transform(docs)
    return vectorizer.get_feature_names()  
  
full_vocab = get_vocab(docs)
len(full_vocab)

65354

##### Get Industries + Sectors

In [0]:
sectors=df_metadata["Sector"].dropna().drop_duplicates().values
industries=df_metadata["Industry"].dropna().drop_duplicates().values
sectors_industries = df_metadata.dropna(subset=["Sector","Industry"]).groupby(["Sector","Industry"]).agg({"ticker":pd.Series.nunique})

<br>
### Build Generic Earnings Call Stop Word Sets at Call/Paragraph Levels
* Words that are in 75% of calls
* Words that are in 7.5% of paragraphs

In [9]:
# build high-level stop word list at the entire call level
docs = df_metadata["Prepared Remarks Text"]
vectorizer = CountVectorizer(binary=True, lowercase=True, min_df=0.75)
vectors = vectorizer.fit_transform(docs)
feature_names = vectorizer.get_feature_names()
results = pd.DataFrame(data=vectors.toarray(), index=docs.index, columns=vectorizer.get_feature_names())
call_stopwords = results.sum(axis=0)
call_stopwords = list(call_stopwords.index)
print(len(call_stopwords))
print(np.array(call_stopwords))
print()

# build high-level stop word list at the paragraph level
docs = df_text_prepremarks["Text"]
vectorizer = CountVectorizer(binary=True, lowercase=True, min_df=0.075)
vectors = vectorizer.fit_transform(docs)
feature_names = vectorizer.get_feature_names()
results = pd.DataFrame(data=vectors.toarray(), index=docs.index, columns=vectorizer.get_feature_names())
paragraph_stopwords = results.sum(axis=0)
paragraph_stopwords = list(paragraph_stopwords.index)
print(len(paragraph_stopwords))
print(np.array(paragraph_stopwords))

# combine stopwords from both levels
combined_stopwords = set(paragraph_stopwords).union(set(call_stopwords))

161
['10' '12' '20' '2017' '2018' '2019' 'about' 'across' 'actual' 'addition'
 'additional' 'all' 'also' 'an' 'and' 'any' 'approximately' 'are' 'as'
 'at' 'back' 'based' 'basis' 'be' 'been' 'before' 'believe' 'both'
 'business' 'but' 'by' 'call' 'can' 'capital' 'cash' 'company' 'compared'
 'continue' 'continued' 'cost' 'current' 'driven' 'due' 'during'
 'earnings' 'end' 'everyone' 'expect' 'expected' 'financial' 'first' 'for'
 'forward' 'fourth' 'from' 'full' 'future' 'gaap' 'good' 'growth' 'had'
 'has' 'have' 'high' 'higher' 'impact' 'in' 'including' 'income'
 'increase' 'increased' 'into' 'is' 'it' 'joining' 'last' 'like' 'line'
 'll' 'long' 'looking' 'lower' 'made' 'margin' 'market' 'may' 'me'
 'million' 'more' 'most' 'net' 'new' 'non' 'not' 'now' 'of' 'on' 'one'
 'operating' 'or' 'other' 'our' 'over' 'per' 'performance' 'primarily'
 'prior' 'provide' 'quarter' 'questions' 'range' 'rate' 're' 'related'
 'release' 'result' 'results' 'revenue' 'sales' 'second' 'see' 'share'
 'signific

<br>
### Word Frequency Functions
*   This is where the magic happens.



#### Helpers
Many of these are not currently used, but were at one point useful during development.

In [0]:
def get_calls(from_date, to_date, category=None, category_filter=None):
    calls = df_metadata[df_metadata.apply(lambda row: (to_date > row["call date"] >= from_date), axis=1)]
    if category is not None:
        calls = calls[calls[category]==category_filter]
    return calls

def get_calls_for_shared_tickers(calls1, calls2):
    tickers = list(set(calls2["ticker"]).intersection(set(calls1["ticker"])))
    calls1 = calls1[calls1["ticker"].isin(tickers)]
    calls2 = calls2[calls2["ticker"].isin(tickers)]
    return calls1, calls2

def get_calls_interest_reference(interest_date_from, interest_date_to, reference_date_from, referece_date_to, cat_type, cat_filter, shared=False):
    interest_calls = get_calls(interest_date_from, interest_date_to, cat_type, cat_filter)
    reference_calls = get_calls(reference_date_from, referece_date_to, cat_type, cat_filter)
    if shared:
        reference_calls, interest_calls = get_calls_for_shared_tickers(reference_calls, interest_calls)
    return interest_calls, reference_calls 
  
def get_norm_count_vector(docs, index, vocab, stopwords):
    vectorizer = CountVectorizer(binary=False, lowercase=True, vocabulary=vocab, stop_words=stopwords)
    vectors = vectorizer.fit_transform(docs)
    feature_names = vectorizer.get_feature_names()
    results = pd.DataFrame(data=vectors.toarray(), index=index, columns=vectorizer.get_feature_names())
    results=results.div(results.sum(axis=1), axis=0)
    results.fillna(0,inplace=True)
    return results
  
def get_tfidf_vector(docs, index, vocab, stopwords):
    vectorizer = TfidfVectorizer(binary=False, lowercase=True, vocabulary=vocab, stop_words=stopwords)
    vectors = vectorizer.fit_transform(docs)
    feature_names = vectorizer.get_feature_names()
    results = pd.DataFrame(data=vectors.toarray(), index=index, columns=vectorizer.get_feature_names())
    results.fillna(0,inplace=True)
    return results
  
# get top 20 changes up and down
def get_keywords_norm_count(first_qtr_calls, second_qtr_calls):
    # build relative frequency word vector for first period
    docs = first_qtr_calls["Prepared Remarks Text"]
    first_qtr_results = get_norm_count_vector(docs,first_qtr_calls["ticker"], full_vocab, combined_stopwords)

    # build relative frequency word vector for second period
    docs = second_qtr_calls["Prepared Remarks Text"]
    second_qtr_results = get_norm_count_vector(docs,second_qtr_calls["ticker"], full_vocab, combined_stopwords)

    # get the difference in relative frequency by company/word
    diff = (second_qtr_results.mean() - first_qtr_results.mean()).sort_values()
    return diff.head(20).append(diff.tail(20))
  
# get top 20 changes up and down
def get_keywords_tfidf(first_qtr_calls, second_qtr_calls):
    # build relative frequency word vector for first quarter
    docs = first_qtr_calls["Prepared Remarks Text"]
    first_qtr_results = get_tfidf_vector(docs, first_qtr_calls["ticker"], full_vocab, combined_stopwords)

    # build relative frequency word vector for second quarter
    docs = second_qtr_calls["Prepared Remarks Text"]
    second_qtr_results = get_tfidf_vector(docs, second_qtr_calls["ticker"], full_vocab, combined_stopwords)

    # get the difference in relative frequency by company/word
    diff = (second_qtr_results.mean() - first_qtr_results.mean()).sort_values()
    return diff.head(20), diff.tail(20).sort_values(ascending=False)


def get_count_vector(docs, index, vocab, stopwords):
    vectorizer = CountVectorizer(binary=False, lowercase=True, vocabulary=vocab, stop_words=stopwords)
    vectors = vectorizer.fit_transform(docs)
    feature_names = vectorizer.get_feature_names()
    results = pd.DataFrame(data=vectors.toarray(), index=index, columns=vectorizer.get_feature_names())
    results.fillna(0,inplace=True)
    return results

# get top 20 changes up and down
def get_keywords_count(first_qtr_calls, second_qtr_calls, additional_stopwords=[]): 
    # build relative frequency word vector for first period
    docs = first_qtr_calls["Prepared Remarks Text"]
    first_qtr_results = get_count_vector(docs,first_qtr_calls["ticker"], 
                                         full_vocab, list(combined_stopwords)+additional_stopwords)

    # build relative frequency word vector for second period
    docs = second_qtr_calls["Prepared Remarks Text"]
    second_qtr_results = get_count_vector(docs,second_qtr_calls["ticker"], 
                                          full_vocab, list(combined_stopwords)+additional_stopwords)

    # get the difference in relative frequency by company/word
    diff = (second_qtr_results.sum() - first_qtr_results.sum()).sort_values()
    return diff.head(20), diff.tail(20).sort_values(ascending=False)

#### Primary Functions

In [0]:
def get_stopwords_for_all_calls_during_time_period(interest_date_from, interest_date_to,relative_freq_to_max=0.5):
    calls = get_calls(interest_date_from, interest_date_to)
    docs=calls["Prepared Remarks Text"]
    all_results = get_count_vector(docs,calls["ticker"], full_vocab, combined_stopwords)
    all_results = all_results.sum().sort_values(ascending=False)
    num=all_results[0]*relative_freq_to_max
    all_results = list(all_results[all_results>num].index)
    return all_results
    
def get_keywords_by_comparison(interest_date_from, interest_date_to, reference_date_from, referece_date_to, 
                               cat_type, cat_filter, 
                               filter_reference=True, shared=False, additional_stopwords=None, 
                               method="counts", number_words_to_return=None):
  
    # build the sets of calls to compare
    interest_calls = get_calls(interest_date_from, interest_date_to, cat_type, cat_filter)
    
    
    # should the reference period use the same filter
    if filter_reference:
      reference_calls = get_calls(reference_date_from, referece_date_to, cat_type, cat_filter)
    else:
      reference_calls = get_calls(reference_date_from, referece_date_to, None, None)
    
    # remove interest calls from the reference calls
    reference_calls = reference_calls[~(reference_calls["Link"].isin(interest_calls["Link"]))]
    
    # filter to calls with tickers in both reference and interest
    if shared:
        reference_calls, interest_calls = get_calls_for_shared_tickers(reference_calls, interest_calls)
    
    
    # determine what method will be used to identify changes
    if method=="counts":
        docs=reference_calls["Prepared Remarks Text"]
        reference_results = get_count_vector(docs,reference_calls["ticker"], full_vocab, combined_stopwords)
        reference_results = reference_results.sum().sort_values(ascending=False)

        docs=interest_calls["Prepared Remarks Text"]
        interest_results = get_count_vector(docs, interest_calls["ticker"], full_vocab, combined_stopwords)
        interest_results = interest_results.sum().sort_values(ascending=False)
        
        # convert counts to percent of total and get difference
        interest_results_pct=interest_results.div(interest_results.sum())
        reference_results_pct=reference_results.div(reference_results.sum())
        diff = interest_results_pct - reference_results_pct        
    elif method == "tfidf":
        reference_doc=" ".join(reference_calls["Prepared Remarks Text"])
        interest_doc=" ".join(interest_calls["Prepared Remarks Text"])
        docs = [reference_doc,interest_doc]

        vectorizer = TfidfVectorizer(binary=False, lowercase=True, stop_words=combined_stopwords)
        vectors = vectorizer.fit_transform(docs)
        feature_names = vectorizer.get_feature_names()
        results = pd.DataFrame(data=vectors.toarray(), columns=vectorizer.get_feature_names())
        results.fillna(0,inplace=True)
        diff = results.loc[1]-results.loc[0]
    
    
    # drop words from results
    if additional_stopwords is not None:
        diff=diff.drop(additional_stopwords, errors="ignore")
    
    diff = diff.sort_values(ascending=False)
    if number_words_to_return is not None:  
        diff = diff.head(number_words_to_return)
    
    return diff, interest_calls

### Build Paragraph List (and Word Vector) for Period of Interest

In [0]:
period_stopwords = get_stopwords_for_all_calls_during_time_period(interest_date_from, interest_date_to, 0.05)

new_topics, interest_calls = get_keywords_by_comparison(interest_date_from, interest_date_to, reference_date_from, referece_date_to, 
                                                        cat_type, cat_filter, filter_reference=True, shared=True, 
                                                        additional_stopwords=period_stopwords, method="tfidf", number_words_to_return=25)

In [0]:
this_period_paragraphs = df_text_prepremarks[df_text_prepremarks.apply(lambda row: (interest_date_to > row["call date"] >= interest_date_from), axis=1)]
docs=list(this_period_paragraphs["Text"])
vectorizer = CountVectorizer(binary=True, lowercase=True, vocabulary=new_topics.index)
vectors = vectorizer.fit_transform(docs)
feature_names = vectorizer.get_feature_names()
this_period_vector = pd.DataFrame(data=vectors.toarray(), index=list(this_period_paragraphs["Text"].index), columns=vectorizer.get_feature_names())

<br>
## Summarization and TextRank
Now that you have some keywords, can you identify/compile text to give understanding of why those words are important?
* text-rank summarization using summanlp
  * no parameters:  just gave me the first sentence
  * 50 words: interesting!

In [31]:
for word in new_topics.index[0:1]:   
    print("Word:",word)
    
    paragraphs_with_word = interest_calls[(interest_calls["Prepared Remarks Text"].str.contains(word, flags=re.IGNORECASE, regex=True))]
    unique_calls_with_word_industry = paragraphs_with_word[["company","ticker","Link"]].drop_duplicates()
    print("    Number of Calls with Word in Group This Period:",len(unique_calls_with_word_industry))
    print("        Tickers with Word in Group This Period:",set(unique_calls_with_word_industry["ticker"].drop_duplicates()))
    
    paragraphs_with_word_all_calls_this_period = this_period_paragraphs.loc[this_period_vector[this_period_vector[word]==1].index]
    unique_calls_with_word_this_period = paragraphs_with_word_all_calls_this_period[["company","ticker","Link"]].drop_duplicates()
    print("    Number of Calls with Word This Period (All Calls):",len(unique_calls_with_word_this_period))
    print("        Tickers with Word This Period (All Calls):",set(unique_calls_with_word_this_period["ticker"].drop_duplicates()))

    print()
    for _, row in unique_calls_with_word_this_period.iterrows():
        print(row["company"])
        print(row["Link"])
        text = list(paragraphs_with_word_all_calls_this_period[paragraphs_with_word_all_calls_this_period["Link"]==row["Link"]]["Text"])
        t = " ".join(text)
        
        if len(text) < 5:
            print("Summary at Call Level")
            pprint.pprint(summarizer.summarize(t, words=100))
        else:
            print("Summary of Each Paragraph")
            for t in text:
                summary = summarizer.summarize(t, words=25)
                if summary: pprint.pprint(summary); print()
        print()
    print()

Word: gymboree
    Number of Calls with Word in Group This Period: 1
        Tickers with Word in Group This Period: {'PLCE'}
    Number of Calls with Word This Period (All Calls): 7
        Tickers with Word This Period (All Calls): {'RPAI', 'PLCE', 'TCO', 'RPT', 'VRS', 'SKT', 'RILY'}

Retail Properties of America Inc
https://www.fool.com/earnings/call-transcripts/2019/02/13/retail-properties-of-america-inc-rpai-q4-2018-earn.aspx
Summary at Call Level
('As it relates to our expectations for same-store and leasing trends for '
 '2019, we have seen several smaller bankruptcies this year, including Shopko '
 'Gymboree, things remembered beauty brands and Charlotte Russe.\n'
 'And while we have been relatively unaffected to-date, we remain cautious on '
 'our watch list and same-store NOI growth assumption.\n'
 'In addition to our 50 basis points of bad debt embedded in that assumption, '
 'we have assumed that a small number of our watch list tenants do not renew '
 'certain locations th

<br>
## Cross-document Structure Theory
Now that you have some keywords, can you identify/compile text to give understanding of why those words are important?